In [1]:
import datasets
import os
import numpy as np
import sys
sys.path.append("../retrievers")

from tqdm import tqdm
from totto_retriever import TottoRetriever
from retriever import Retriever



In [2]:
os.environ["HF_HOME"] = "/projects/ogma2/users/andrewsi/cache/huggingface/"
root = "/projects/ogma2/users/andrewsi/control-data2text"

In [3]:
train_dataset = f"{root}/prototype-retrieval/datasets/totto/totto_train_v2"
val_dataset = f"{root}/prototype-retrieval/datasets/totto/totto_validation"
totto_val_json = f"{root}/transformers/examples/seq2seq/test_data/totto/val_with_headers_only.json"

retriever = TottoRetriever(val_dataset)

In [4]:
retriever.dataset

Dataset({
    features: ['source', 'source_headers_only', 'source_headers_only_embed', 'target'],
    num_rows: 7700
})

In [5]:
src = retriever.dataset[0]["source"]
tgt = retriever.dataset[0]["target"]

In [6]:
retriever.__class__.mask_target(src, tgt)

'[MASK] was the [MASK] [MASK] of [MASK] from [MASK] .'

In [5]:
retriever.add_masked_targets()

In [7]:
retriever.dataset

Dataset({
    features: ['source', 'source_headers_only', 'source_headers_only_embed', 'target'],
    num_rows: 7700
})

In [ ]:
retriever.add_embeds("source_headers_only", gpu=1)

In [ ]:
retrieval_k = 10
weighted = False
max_edit_dist = 5
retriever.write_train_set(f"/projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/totto_proto/train_k{retrieval_k}{'_max' + str(max_edit_dist) if max_edit_dist else ''}{'_weighted' if weighted else ''}.json", retrieval_k=retrieval_k, max_edit_dist=max_edit_dist, weighted=weighted)

In [6]:
retriever.write_eval_set("/projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/totto_eval/val_from_train_headers_only.json", retrieval_embed="source_headers_only_embed", retrieval_path=train_dataset, eval_k=5, gpu=1)

100%|██████████| 7700/7700 [15:19<00:00,  8.38it/s]
Wrote eval data to file: /projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/totto_eval/val_from_train_headers_only.json


In [8]:
retriever.dataset

Dataset({
    features: ['source', 'source_headers_only', 'source_headers_only_embed', 'target'],
    num_rows: 7700
})

In [ ]:
TottoRetriever.mask_target(retriever.dataset[0]["source"], retriever.dataset[0]["target"])

In [ ]:
val_data = retriever.dataset
val_data.add_faiss_index("source_headers_only_embed")

In [ ]:
idx = 1502

results = val_data.get_nearest_examples("source_headers_only_embed", np.array(val_data[idx]["source_headers_only_embed"], dtype=np.float32), k=30)

print(val_data[idx]["target"])
print("===================================")

for i, target in enumerate(results[1]["target"]):
        print("({}) {}".format(results[0][i], target))

In [ ]:
train_dataset = datasets.load_from_disk("../datasets/totto/totto_train_v2")

In [ ]:
train_dataset

In [ ]:
train_dataset = train_dataset.map(lambda x: x, remove_columns=['edit_dist_map'])

In [ ]:
train_dataset.save_to_disk("../datasets/totto/totto_train_v2")

In [ ]:
train_dataset

In [ ]:
totto_dataset = datasets.load_dataset("json", data_files={"train": f"{root}/transformers/examples/seq2seq/test_data/totto_proto/train_k10_weighted.json"})

In [ ]:
list(totto_dataset['train'].features.keys())